<a href="https://colab.research.google.com/github/iam-nhi-nguyen/Massp2019/blob/master/transfermarkt/transfermarkt_soccer_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrape soccer game html from transfermarkt

*Written by Nhi Nguyen*

June 1, 2021

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import gc
from multiprocessing import Pool
from functools import reduce
import re

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_url = "https://www.transfermarkt.com"
headers = {"User-Agent": 
           "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"}

## General function

In [ ]:
def get_html(tablename, df, filename, v=None):
    start_time = time.time()
    n = df.shape[0]
    maxrow = 1000
    if v:
        print(f"\tGet {tablename} html {v:02}: ")
        lenrange = min(n,v*maxrow) - (v-1)*maxrow
        for i in range((v-1)*maxrow, min(n,v*maxrow)):
            url = df["url"][i]
            html = requests.get(url, headers=headers)
            df["content"][i] = str(BeautifulSoup(html.content, "html.parser"))
            p = round(100*(i+1-(v-1)*maxrow)/lenrange, 2)
            print(f"\r{'='*round(p/2)+' '*(50-round(p/2))} {p: 6}%\t| {round(time.time() - start_time): 5} seconds", end="")
        print()
        df.iloc[(v-1)*maxrow:min(n,v*maxrow),].to_csv(f"{filename}_{v:03}.csv", index=False)

## Competitions

In [ ]:
competitions_html = pd.DataFrame(columns=["id","name","url","format","content"])

In [ ]:
def get_competitions_url(continent):
    global competitions_html
    df = []
    
    # get continential competitions
    if continent != "fifa":
        # get max page
        page = f"{base_url}/wettbewerbe/{continent}"
        html = requests.get(page, headers=headers)
        soup = BeautifulSoup(html.content, "html.parser")
        lastpage = soup.find("li", {"class": "letzte-seite"})
        pagecnt = int(lastpage.find("a").get("href").split("=")[-1]) if lastpage else 1
        # go through all pages
        for i in range(1, pagecnt+1):
            page = f"{base_url}/wettbewerbe/{continent}?page={i}"
            html = requests.get(page, headers=headers)
            soup = BeautifulSoup(html.content, "html.parser")
            rows = soup.find_all("table", {"class": "inline-table"})
            for row in rows:
                info = row.find_all("a")[-1]
                name = info.get("title")
                url = f"{base_url}{info.get('href')}"
                id = url.split('/')[-1]
                df.append([id, name, url])
    
    # get international cups
    page = f"{base_url}/wettbewerbe/{continent}"
    html = requests.get(page, headers=headers)
    soup = BeautifulSoup(html.content, "html.parser")
    cups = soup.find_all("div", {"class": "clearer relevante-wettbewerbe-auflistung"})
    cups = [item for sublist in list(map(lambda x: x.find_all("a"), cups)) for item in sublist]
    for cup in cups:
        name = cup.get("title")
        url = f"{base_url}{cup.get('href')}"
        id = url.split('/')[-1]
        df.append([id, name, url])
    
    competitions_html = competitions_html.append(pd.DataFrame(df, columns=["id","name","url"]), ignore_index=True)

In [ ]:
print("Get competitions url: ", end="")
start_time = time.time()
continents = ["europa", "asien", "amerika", "afrika", "fifa"]
for continent in continents:
    get_competitions_url(continent)
competitions_html = competitions_html.drop_duplicates(subset="url").reset_index(drop=True)
print(f"{round(time.time() - start_time, 4)} seconds")

```
Get competitions url: 79.2558 seconds
```

In [ ]:
competitions_html = get_html("competitions", competitions_html, "./drive/MyDrive/transfermarkt/1-competitions/competitions_html")

```
Get competitions html: 
==================================================  100.0%	|  2509 seconds
Finished in 2509
```



In [ ]:
print("Get competitions html format: ", end="")
start_time = time.time()
n = competitions_html.shape[0]
for i in range(n):
    soup = BeautifulSoup(competitions_html["content"][i], "lxml")
    if soup.find('div', {'class': 'box-personeninfos'}) is not None:
        competitions_html["format"][i] = 1
    elif soup.find('div', {'class': 'dataContent'}) is not None:
        competitions_html["format"][i] = 2
    else:
        competitions_html["format"][i] = 0
competitions_html.to_csv(f"./drive/MyDrive/transfermarkt/1-competitions/competitions_html.csv", index=False)
print(f"{round(time.time() - start_time)} seconds")

```
Get competitions html format: 50.9101 seconds
```

In [ ]:
competitions_html.shape

```
(912, 5)
```

## Campaigns

In [ ]:
campaigns_html = pd.DataFrame(columns=["id","competition_id","year","url","format","content"])

In [ ]:
def get_campaigns_url(ind):
    global campaigns_html
    global competitions_html
    df = []
    
    competition = competitions_html.iloc[ind]
    url = competition["url"]
    competition_id = competition["id"]
    content = competition["content"]
    format = competition["format"]

    soup = BeautifulSoup(content, "lxml")
    years = soup.find("table", {"class": "auflistung"})
    
    # get list of years
    if years is not None:
        years = years.find_all("option")
        years = list(map(lambda tag: tag.get("value"), years))
    # extract year from the competition title
    else:
        title = competition["url"].split("/")[-4].split("-")
        if title[-1].isdigit() and len(title[-1]) == 4:
            years = [title[-1]]
        elif title[0].isdigit() and len(title[0]) == 4:
            years = [title[0]]
        else:
            years = [""]
    # construct url
    if competition["format"] == 1:
        sub = "gesamtspielplan/wettbewerb"
    else:
        sub = "startseite/pokalwettbewerb"
    for year in years:
        campaign_url = url.replace('startseite/wettbewerb', sub) + '/saison_id/' + year
        df.append([competition_id + '_' + year, competition_id, year, campaign_url, format])
    
    # appeand results
    campaigns_html = campaigns_html.append(pd.DataFrame(df, columns=["id","competition_id","year","url","format"]), ignore_index=True)

In [ ]:
print("Get campaigns url: ", end="")
start_time = time.time()
competitions_html = pd.read_csv("./drive/MyDrive/transfermarkt/1-competitions/competitions_html.csv")
n = competitions_html.shape[0]
for i in range(n):
    get_campaigns_url(i)
campaigns_html = campaigns_html.drop_duplicates(subset="url").reset_index(drop=True)
campaigns_html.to_csv("./drive/MyDrive/transfermarkt/2-campaigns/campaigns_html_0.csv", index=False)
print(f"{round(time.time() - start_time)} seconds")

```
Get campaigns url: 54.3616 seconds
```

In [ ]:
campaigns_html.shape

```
(11043, 6)
```

In [ ]:
get_html("campaigns", campaigns_html, "./drive/MyDrive/transfermarkt/2-campaigns/campaigns_html")

```
Get campaigns html
    Get campaigns html 001: ==================================================  100.0%	|  4818 seconds
    Get campaigns html 002: ==================================================  100.0%	|  2762 seconds
    Get campaigns html 003: ==================================================  100.0%	|  1600 seconds
    Get campaigns html 004: ==================================================  100.0%	|  3577 seconds
    Get campaigns html 005: ==================================================  100.0%	|  2854 seconds
    Get campaigns html 006: ==================================================  100.0%	|  4818 seconds
    Get campaigns html 007: ==================================================  100.0%	|  2762 seconds
    Get campaigns html 008: ==================================================  100.0%	|  1600 seconds
    Get campaigns html 009: ==================================================  100.0%	|  3577 seconds
    Get campaigns html 010: ==================================================  100.0%	|  2854 seconds
    Get campaigns html 011: ==================================================  100.0%	|  4818 seconds
    Get campaigns html 012: ==================================================  100.0%	|  2762 seconds
Finished in 38802 seconds
```

## Games

In [ ]:
games_html = pd.DataFrame(columns=["id","campaign_id","url","format","content"])

In [ ]:
def get_games_url(ind):
    global games_html
    global campaigns_html
    df = []
    
    campaign = campaigns_html.iloc[ind]
    url = campaign["url"]
    campaign_id = campaign["id"]
    content = campaign["content"]
    format = campaign["format"]
    
    soup = BeautifulSoup(content, "lxml")
    
    if format == 1:
        tags = soup.find_all('a', {'class': 'ergebnis-link'})
    elif format == 2:
        tags = soup.find_all('a', {'title': 'Match report'})
    else:
        return
    for tag in tags:
        df.append([campaign_id + "_" + tag.get('href').split('/')[-1], campaign_id, base_url + tag.get('href'), format])
    
    games_html = games_html.append(pd.DataFrame(df, columns=["id","campaign_id","url","format"]), ignore_index=True)

In [ ]:
print("Get games url: ", end="")
start_time = time.time()
nv = 12
for v in range(1,nv+1):
    campaigns_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/2-campaigns/campaigns_html_{v:03}.csv")
    for i in range(campaigns_html.shape[0]):
        get_games_url(i)
games_html = games_html.drop_duplicates(subset="url").reset_index(drop=True)
games_html.to_csv("./drive/MyDrive/transfermarkt/3-games/games_html_0.csv", index=False)
print(f"{round(time.time() - start_time, 4)} seconds")

```
Get games url: 4056.3852 seconds
```

In [ ]:
games_html.shape

(963104, 5)

```
(957861, 5)
```

```
(955627, 5)
```

In [ ]:
nv = 964
for v in range(1, nv+1):
    get_html("games", games_html, "./drive/MyDrive/transfermarkt/3-games/games_html", v)

```
Finished in 1259700 seconds
```

##### Very long results

```
Get games html:
	Get games html 001: ==================================================  100.0%	|  1221 seconds
	Get games html 002: ==================================================  100.0%	|  1268 seconds
	Get games html 003: ==================================================  100.0%	|  1264 seconds
	Get games html 004: ==================================================  100.0%	|  1134 seconds
	Get games html 005: ==================================================  100.0%	|  2163 seconds
	Get games html 007: ==================================================  100.0%	|  1445 seconds
	Get games html 008: ==================================================  100.0%	|  2109 seconds
	Get games html 009: ==================================================  100.0%	|  1182 seconds
	Get games html 010: ==================================================  100.0%	|  1401 seconds
	Get games html 011: ==================================================  100.0%	|  1066 seconds
	Get games html 012: ==================================================  100.0%	|  1468 seconds
	Get games html 014: ==================================================  100.0%	|  2177 seconds
	Get games html 015: ==================================================  100.0%	|  1180 seconds
	Get games html 016: ==================================================  100.0%	|  1283 seconds
	Get games html 017: ==================================================  100.0%	|  1285 seconds
	Get games html 018: ==================================================  100.0%	|  1230 seconds
	Get games html 019: ==================================================  100.0%	|  1062 seconds
	Get games html 020: ==================================================  100.0%	|  1143 seconds
	Get games html 021: ==================================================  100.0%	|  1368 seconds
	Get games html 022: ==================================================  100.0%	|  1333 seconds
	Get games html 023: ==================================================  100.0%	|  1020 seconds
	Get games html 024: ==================================================  100.0%	|   439 seconds
	Get games html 025: ==================================================  100.0%	|  1244 seconds
	Get games html 026: ==================================================  100.0%	|   915 seconds
	Get games html 027: ==================================================  100.0%	|   998 seconds
	Get games html 028: ==================================================  100.0%	|  1192 seconds
	Get games html 029: ==================================================  100.0%	|   882 seconds
	Get games html 030: ==================================================  100.0%	|   983 seconds
	Get games html 031: ==================================================  100.0%	|  1218 seconds
	Get games html 032: ==================================================  100.0%	|   970 seconds
	Get games html 033: ==================================================  100.0%	|  1897 seconds
	Get games html 034: ==================================================   99.0%	|   979 seconds
	Get games html 035: ==================================================  100.0%	|  1015 seconds
	Get games html 036: ==================================================  100.0%	|   873 seconds
	Get games html 037: ==================================================  100.0%	|  1165 seconds
	Get games html 038: ==================================================  100.0%	|   913 seconds
	Get games html 039: ==================================================  100.0%	|  2076 seconds
	Get games html 040: ==================================================  100.0%	|  2165 seconds
	Get games html 041: ==================================================  100.0%	|   612 seconds
	Get games html 042: ==================================================  100.0%	|  2193 seconds
	Get games html 043: =================================================  100.0%	|  1121 seconds
	Get games html 044: ==================================================  100.0%	|  1244 seconds
	Get games html 045: ==================================================  100.0%	|  1404 seconds
	Get games html 046: ==================================================  100.0%	|  2138 seconds
	Get games html 047: ==================================================  100.0%	|  1428 seconds
	Get games html 048: ==================================================  100.0%	|  1089 seconds
	Get games html 049: ==================================================  100.0%	|  1192 seconds
	Get games html 050: ==================================================  100.0%	|  1110 seconds
	Get games html 051: ==================================================  100.0%	|  1144 seconds
	Get games html 052: ==================================================  100.0%	|  1421 seconds
	Get games html 053: ==================================================  100.0%	|  1146 seconds
	Get games html 054: ==================================================  100.0%	|  1037 seconds
	Get games html 055: ==================================================  100.0%	|   971 seconds
	Get games html 056: ==================================================  100.0%	|  1054 seconds
	Get games html 057: ==================================================  100.0%	|   900 seconds
	Get games html 058: ==================================================  100.0%	|  1175 seconds
	Get games html 059: ==================================================  100.0%	|   677 seconds
	Get games html 060: ==================================================  100.0%	|  1469 seconds
	Get games html 061: ==================================================  100.0%	|  1139 seconds
	Get games html 062: ==================================================  100.0%	|  1222 seconds
	Get games html 063: ==================================================  100.0%	|  1466 seconds
	Get games html 064: ==================================================  100.0%	|  1140 seconds
	Get games html 065: ==================================================  100.0%	|  1249 seconds
	Get games html 066: ==================================================  100.0%	|  1519 seconds
	Get games html 067: ==================================================  100.0%	|  1460 seconds
	Get games html 068: ==================================================  100.0%	|  1415 seconds
	Get games html 069: ==================================================  100.0%	|  2126 seconds
	Get games html 070: ==================================================  100.0%	|  1390 seconds
	Get games html 071: ==================================================  100.0%	|   603 seconds
	Get games html 072: ==================================================  100.0%	|  1081 seconds
	Get games html 073: ==================================================  100.0%	|  1062 seconds
	Get games html 074: ==================================================  100.0%	|  1146 seconds
	Get games html 075: ==================================================  100.0%	|  1054 seconds
	Get games html 076: ==================================================  100.0%	|  1221 seconds
	Get games html 077: ==================================================  100.0%	|  1237 seconds
	Get games html 078: ==================================================  100.0%	|  1144 seconds
	Get games html 079: ==================================================  100.0%	|  1233 seconds
	Get games html 080: ==================================================  100.0%	|  1272 seconds
	Get games html 081: ==================================================  100.0%	|  1125 seconds
	Get games html 082: ==================================================  100.0%	|  2147 seconds
	Get games html 083: ==================================================  100.0%	|  2147 seconds
	Get games html 084: ==================================================  100.0%	|  1402 seconds
	Get games html 085: ==================================================  100.0%	|  1446 seconds
	Get games html 086: ==================================================  100.0%	|  1418 seconds
	Get games html 087: ==================================================  100.0%	|   598 seconds
	Get games html 088: ==================================================  100.0%	|  1176 seconds
	Get games html 089: ==================================================  100.0%	|  1252 seconds
	Get games html 090: ==================================================  100.0%	|  1161 seconds
	Get games html 091: ==================================================  100.0%	|  1305 seconds
	Get games html 092: ==================================================  100.0%	|  1438 seconds
	Get games html 093: ==================================================  100.0%	|  2160 seconds
	Get games html 094: ==================================================  100.0%	|  1495 seconds
	Get games html 095: ==================================================  100.0%	|  1148 seconds
	Get games html 096: ==================================================  100.0%	|  1146 seconds
	Get games html 097: ==================================================  100.0%	|  1267 seconds
	Get games html 098: ==================================================  100.0%	|  1131 seconds
	Get games html 099: ==================================================  100.0%	|  1442 seconds
	Get games html 100: ==================================================  100.0%	|  1418 seconds
	Get games html 101: ==================================================  100.0%	|  1102 seconds
	Get games html 102: ==================================================  100.0%	|   622 seconds
	Get games html 103: ==================================================  100.0%	|  1196 seconds
	Get games html 103: ==================================================  100.0%	|  1472 seconds
	Get games html 104: ==================================================  100.0%	|  1200 seconds
	Get games html 105: ==================================================  100.0%	|  1213 seconds
	Get games html 106: ==================================================  100.0%	|  1421 seconds
	Get games html 107: ==================================================  100.0%	|  1238 seconds
	Get games html 108: ==================================================  100.0%	|  1284 seconds
	Get games html 109: ==================================================  100.0%	|  1159 seconds
	Get games html 110: ==================================================  100.0%	|  2179 seconds
	Get games html 111: ==================================================  100.0%	|  2163 seconds
	Get games html 112: ==================================================  100.0%	|   634 seconds
	Get games html 113: ==================================================  100.0%	|  1109 seconds
	Get games html 114: ==================================================  100.0%	|  1443 seconds
	Get games html 115: ==================================================  100.0%	|  1515 seconds
	Get games html 116: ==================================================  100.0%	|  1512 seconds
	Get games html 117: ==================================================  100.0%	|  1186 seconds
	Get games html 118: ==================================================  100.0%	|  1234 seconds
	Get games html 119: ==================================================  100.0%	|  1333 seconds
	Get games html 120: ==================================================  100.0%	|  1167 seconds
	Get games html 121: ==================================================  100.0%	|  1165 seconds
	Get games html 122: ==================================================  100.0%	|  1161 seconds
	Get games html 123: ==================================================  100.0%	|  1114 seconds
	Get games html 124: ==================================================  100.0%	|  1256 seconds
	Get games html 125: ==================================================  100.0%	|  1473 seconds
	Get games html 126: ==================================================  100.0%	|  1491 seconds
	Get games html 127: ==================================================  100.0%	|  1218 seconds
	Get games html 128: ==================================================  100.0%	|  1437 seconds
	Get games html 129: ==================================================  100.0%	|  2146 seconds
	Get games html 130: ==================================================  100.0%	|  1141 seconds
	Get games html 131: ==================================================  100.0%	|  2191 seconds
	Get games html 132: ==================================================  100.0%	|   668 seconds
	Get games html 133: ==================================================  100.0%	|  1486 seconds
	Get games html 134: ==================================================  100.0%	|  1473 seconds
	Get games html 135: ==================================================  100.0%	|  1237 seconds
	Get games html 136: ==================================================  100.0%	|  1241 seconds
	Get games html 137: ==================================================  100.0%	|  1426 seconds
	Get games html 138: ==================================================  100.0%	|  1259 seconds
	Get games html 138: ==================================================  100.0%	|  2198 seconds
	Get games html 139: ==================================================  100.0%	|  1108 seconds
	Get games html 140: ==================================================  100.0%	|  1247 seconds
	Get games html 141: ==================================================  100.0%	|  1295 seconds
	Get games html 142: ==================================================  100.0%	|  1150 seconds
	Get games html 143: ==================================================  100.0%	|  1449 seconds
	Get games html 144: ==================================================  100.0%	|  1030 seconds
	Get games html 145: ==================================================  100.0%	|   748 seconds
	Get games html 146: ==================================================  100.0%	|  1548 seconds
	Get games html 147: ==================================================  100.0%	|  1238 seconds
	Get games html 148: ==================================================  100.0%	|  1306 seconds
	Get games html 149: ==================================================  100.0%	|  1543 seconds
	Get games html 150: ==================================================  100.0%	|  1371 seconds
	Get games html 151: ==================================================  100.0%	|  1194 seconds
	Get games html 152: ==================================================  100.0%	|  1619 seconds
	Get games html 153: ==================================================  100.0%	|  2822 seconds
	Get games html 154: ==================================================  100.0%	|  1610 seconds
	Get games html 155: ==================================================  100.0%	|  1234 seconds
	Get games html 156: ==================================================  100.0%	|  1337 seconds
	Get games html 157: ==================================================  100.0%	|  1285 seconds
	Get games html 158: ==================================================  100.0%	|  1217 seconds
	Get games html 159: ==================================================  100.0%	|  1281 seconds
	Get games html 160: ==================================================  100.0%	|  2141 seconds
	Get games html 161: ==================================================  100.0%	|  2098 seconds
	Get games html 162: ==================================================  100.0%	|  2908 seconds
	Get games html 163: ==================================================  100.0%	|  2015 seconds
	Get games html 164: ==================================================  100.0%	|  2951 seconds
	Get games html 165: ==================================================  100.0%	|  1243 seconds
	Get games html 166: ==================================================  100.0%	|  2172 seconds
	Get games html 167: ==================================================  100.0%	|  3027 seconds
	Get games html 168: ==================================================  100.0%	|  2102 seconds
	Get games html 169: ==================================================  100.0%	|  1989 seconds
	Get games html 170: ==================================================  100.0%	|  1942 seconds
	Get games html 170: ==================================================  100.0%	|  1971 seconds
	Get games html 171: ==================================================  100.0%	|   991 seconds
	Get games html 172: ==================================================  100.0%	|  2486 seconds
	Get games html 173: ==================================================  100.0%	|  2509 seconds
	Get games html 174: ==================================================  100.0%	|  2246 seconds
	Get games html 175: ==================================================  100.0%	|  2306 seconds
	Get games html 176: ==================================================  100.0%	|  2915 seconds
	Get games html 177: ==================================================  100.0%	|  2414 seconds
	Get games html 178: ==================================================  100.0%	|  2285 seconds
	Get games html 179: ==================================================  100.0%	|  2356 seconds
	Get games html 180: ==================================================  100.0%	|  2372 seconds
	Get games html 181: ==================================================  100.0%	|  2404 seconds
	Get games html 182: ==================================================  100.0%	|  2328 seconds
	Get games html 183: ==================================================  100.0%	|  2258 seconds
	Get games html 184: ==================================================  100.0%	|  2285 seconds
	Get games html 185: ==================================================  100.0%	|  2300 seconds
	Get games html 186: ==================================================  100.0%	|  2367 seconds
	Get games html 187: ==================================================  100.0%	|  2327 seconds
	Get games html 188: ==================================================  100.0%	|  1498 seconds
	Get games html 189: ==================================================  100.0%	|  1965 seconds
	Get games html 190: ==================================================  100.0%	|  1963 seconds
	Get games html 191: ==================================================  100.0%	|  1975 seconds
	Get games html 192: ==================================================  100.0%	|  1944 seconds
	Get games html 193: ==================================================  100.0%	|  1894 seconds
	Get games html 194: ==================================================  100.0%	|  2597 seconds
	Get games html 195: ==================================================  100.0%	|  1866 seconds
	Get games html 196: ==================================================  100.0%	|  1831 seconds
	Get games html 197: ==================================================  100.0%	|  2510 seconds
	Get games html 198: ==================================================  100.0%	|  1557 seconds
	Get games html 200: ==================================================  100.0%	|  1823 seconds
	Get games html 201: ==================================================  100.0%	|  2478 seconds
	Get games html 202: ==================================================  100.0%	|  2518 seconds
	Get games html 203: ==================================================  100.0%	|  1099 seconds
	Get games html 204: ==================================================  100.0%	|  1862 seconds
	Get games html 205: ==================================================  100.0%	|  1582 seconds
	Get games html 206: ==================================================  100.0%	|  1632 seconds
	Get games html 207: ==================================================  100.0%	|  2531 seconds
	Get games html 208: ==================================================  100.0%	|  1839 seconds
	Get games html 209: ==================================================  100.0%	|  1500 seconds
	Get games html 210: ==================================================  100.0%	|  1628 seconds
	Get games html 211: ==================================================  100.0%	|  1824 seconds
	Get games html 212: ==================================================  100.0%	|  1543 seconds
	Get games html 213: ==================================================  100.0%	|  1840 seconds
	Get games html 214: ==================================================  100.0%	|  1782 seconds
	Get games html 215: ==================================================  100.0%	|  1769 seconds
	Get games html 216: ==================================================  100.0%	|  1395 seconds
	Get games html 217: ==================================================  100.0%	|  1463 seconds
	Get games html 218: ==================================================  100.0%	|  1368 seconds
	Get games html 219: ==================================================  100.0%	|  1416 seconds
	Get games html 220: ==================================================  100.0%	|  1491 seconds
	Get games html 221: ==================================================  100.0%	|  1725 seconds
	Get games html 222: ==================================================  100.0%	|  1337 seconds
	Get games html 223: ==================================================  100.0%	|   822 seconds
	Get games html 224: ==================================================  100.0%	|  1590 seconds
	Get games html 225: ==================================================  100.0%	|  1555 seconds
	Get games html 226: ==================================================  100.0%	|  1562 seconds
	Get games html 227: ==================================================  100.0%	|  1604 seconds
	Get games html 228: ==================================================  100.0%	|  1586 seconds
	Get games html 229: ==================================================  100.0%	|  1246 seconds
	Get games html 230: ==================================================  100.0%	|  1318 seconds
	Get games html 231: ==================================================  100.0%	|  1264 seconds
	Get games html 232: ==================================================  100.0%	|  1342 seconds
	Get games html 233: ==================================================  100.0%	|  1237 seconds
	Get games html 234: ==================================================  100.0%	|  1361 seconds
	Get games html 235: ==================================================  100.0%	|  2197 seconds
	Get games html 236: ==================================================  100.0%	|  2250 seconds
	Get games html 237: ==================================================  100.0%	|  2233 seconds
	Get games html 238: ==================================================  100.0%	|  2215 seconds
	Get games html 239: ==================================================  100.0%	|   744 seconds
	Get games html 240: ==================================================  100.0%	|  1506 seconds
	Get games html 241: ==================================================  100.0%	|  2206 seconds
	Get games html 242: ==================================================  100.0%	|  1556 seconds
	Get games html 243: ==================================================  100.0%	|  1554 seconds
	Get games html 244: ==================================================  100.0%	|  1525 seconds
	Get games html 245: ==================================================  100.0%	|  1505 seconds
	Get games html 246: ==================================================  100.0%	|  1550 seconds
	Get games html 247: ==================================================  100.0%	|  1238 seconds
	Get games html 248: ==================================================  100.0%	|   680 seconds
	Get games html 249: ==================================================  100.0%	|  1186 seconds
	Get games html 250: ==================================================  100.0%	|  1180 seconds
	Get games html 251: ==================================================  100.0%	|  1209 seconds
	Get games html 252: ==================================================  100.0%	|  1535 seconds
	Get games html 253: ==================================================  100.0%	|  1255 seconds
	Get games html 254: ==================================================  100.0%	|  1189 seconds
	Get games html 254: ==================================================  100.0%	|  1249 seconds
	Get games html 255: ==================================================  100.0%	|  1174 seconds
	Get games html 256: ==================================================  100.0%	|  1124 seconds
	Get games html 257: ==================================================  100.0%	|  1494 seconds
	Get games html 258: ==================================================  100.0%	|  1475 seconds
	Get games html 259: ==================================================  100.0%	|  1454 seconds
	Get games html 260: ==================================================  100.0%	|  1302 seconds
	Get games html 261: ==================================================  100.0%	|  1437 seconds
	Get games html 262: ==================================================  100.0%	|  1437 seconds
	Get games html 263: ==================================================  100.0%	|   712 seconds
	Get games html 264: ==================================================  100.0%	|  1071 seconds
	Get games html 265: ==================================================  100.0%	|  1046 seconds
	Get games html 266: ==================================================  100.0%	|  1085 seconds
	Get games html 267: ==================================================  100.0%	|  1158 seconds
	Get games html 268: ==================================================  100.0%	|  1064 seconds
	Get games html 269: ==================================================  100.0%	|  1114 seconds
	Get games html 270: ==================================================  100.0%	|   574 seconds
	Get games html 271: ==================================================  100.0%	|  1364 seconds
	Get games html 272: ==================================================  100.0%	|  1121 seconds
	Get games html 273: ==================================================  100.0%	|  1410 seconds
	Get games html 274: ==================================================  100.0%	|  1078 seconds
	Get games html 275: ==================================================  100.0%	|  1332 seconds
	Get games html 276: ==================================================  100.0%	|  1148 seconds
	Get games html 277: ==================================================  100.0%	|  1062 seconds
	Get games html 278: ==================================================  100.0%	|  1326 seconds
	Get games html 279: ==================================================  100.0%	|  1354 seconds
	Get games html 280: ==================================================  100.0%	|  1468 seconds
	Get games html 281: ==================================================  100.0%	|   671 seconds
	Get games html 282: ==================================================  100.0%	|  1458 seconds
	Get games html 283: ==================================================  100.0%	|  1446 seconds
	Get games html 284: ==================================================  100.0%	|  1414 seconds
	Get games html 285: ==================================================  100.0%	|  1406 seconds
	Get games html 286: ==================================================  100.0%	|  2089 seconds
	Get games html 287: ==================================================  100.0%	|  2097 seconds
	Get games html 288: ==================================================  100.0%	|  1363 seconds
	Get games html 289: ==================================================  100.0%	|  1443 seconds
	Get games html 290: ==================================================  100.0%	|  2126 seconds
	Get games html 291: ==================================================  100.0%	|  2136 seconds
	Get games html 292: ==================================================  100.0%	|   633 seconds
	Get games html 293: ==================================================  100.0%	|  1147 seconds
	Get games html 294: ==================================================  100.0%	|  2128 seconds
	Get games html 295: ==================================================  100.0%	|  1088 seconds
	Get games html 296: ==================================================  100.0%	|   563 seconds
	Get games html 297: ==================================================  100.0%	|  1013 seconds
	Get games html 298: ==================================================  100.0%	|  1157 seconds
	Get games html 299: ==================================================  100.0%	|  1225 seconds
	Get games html 300: ==================================================  100.0%	|  1196 seconds
	Get games html 301: ==================================================  100.0%	|  1477 seconds
	Get games html 302: ==================================================  100.0%	|  2177 seconds
	Get games html 303: ==================================================  100.0%	|  1450 seconds
	Get games html 304: ==================================================  100.0%	|  1441 seconds
	Get games html 305: ==================================================  100.0%	|  2175 seconds
	Get games html 306: ==================================================  100.0%	|  1184 seconds
	Get games html 307: ==================================================  100.0%	|  1446 seconds
	Get games html 308: ==================================================  100.0%	|  1405 seconds
	Get games html 309: ==================================================  100.0%	|  2080 seconds
	Get games html 310: ==================================================  100.0%	|  2107 seconds
	Get games html 311: ==================================================  100.0%	|   556 seconds
	Get games html 312: ==================================================  100.0%	|  1352 seconds
	Get games html 313: ==================================================  100.0%	|  1045 seconds
	Get games html 314: ==================================================  100.0%	|  1107 seconds
	Get games html 315: ==================================================  100.0%	|  1981 seconds
	Get games html 316: ==================================================  100.0%	|  1332 seconds
	Get games html 317: ==================================================  100.0%	|  1009 seconds
	Get games html 318: ==================================================  100.0%	|  1110 seconds
	Get games html 319: ==================================================  100.0%	|  1222 seconds
	Get games html 320: ==================================================  100.0%	|   944 seconds
	Get games html 321: ==================================================  100.0%	|  1212 seconds
	Get games html 322: ==================================================  100.0%	|   962 seconds
	Get games html 323: ==================================================  100.0%	|   941 seconds
	Get games html 324: ==================================================  100.0%	|   534 seconds
	Get games html 325: ==================================================  100.0%	|   968 seconds
	Get games html 326: ==================================================  100.0%	|  1121 seconds
	Get games html 327: ==================================================  100.0%	|  1198 seconds
	Get games html 328: ==================================================  100.0%	|  1440 seconds
	Get games html 329: ==================================================  100.0%	|  1411 seconds
	Get games html 330: ==================================================  100.0%	|  2082 seconds
	Get games html 331: ==================================================  100.0%	|  1365 seconds
	Get games html 332: ==================================================  100.0%	|  1367 seconds
	Get games html 333: ==================================================  100.0%	|  2101 seconds
	Get games html 334: ==================================================  100.0%	|  1118 seconds
	Get games html 335: ==================================================  100.0%	|  1418 seconds
	Get games html 336: ==================================================  100.0%	|  1443 seconds
	Get games html 337: ==================================================  100.0%	|  2128 seconds
	Get games html 338: ==================================================  100.0%	|  2137 seconds
	Get games html 339: ==================================================  100.0%	|   626 seconds
	Get games html 340: ==================================================  100.0%	|  1392 seconds
	Get games html 341: ==================================================  100.0%	|  1100 seconds
	Get games html 342: ==================================================  100.0%	|  1156 seconds
	Get games html 343: ==================================================  100.0%	|  2091 seconds
	Get games html 344: ==================================================  100.0%	|  1423 seconds
	Get games html 345: ==================================================  100.0%	|  1103 seconds
	Get games html 346: ==================================================  100.0%	|  1202 seconds
	Get games html 347: ==================================================  100.0%	|  1391 seconds
	Get games html 348: ==================================================  100.0%	|  1106 seconds
	Get games html 349: ==================================================  100.0%	|  1415 seconds
	Get games html 350: ==================================================  100.0%	|  1112 seconds
	Get games html 351: ==================================================  100.0%	|  1094 seconds
	Get games html 352: ==================================================  100.0%	|   622 seconds
	Get games html 353: ==================================================  100.0%	|  1085 seconds
	Get games html 354: ==================================================  100.0%	|  1099 seconds
	Get games html 355: ==================================================  100.0%	|  1167 seconds
	Get games html 356: ==================================================  100.0%	|  1395 seconds
	Get games html 357: ==================================================  100.0%	|  1459 seconds
	Get games html 358: ==================================================  100.0%	|  2129 seconds
	Get games html 359: ==================================================  100.0%	|  1399 seconds
	Get games html 360: ==================================================  100.0%	|  1397 seconds
	Get games html 361: ==================================================  100.0%	|  2099 seconds
	Get games html 362: ==================================================  100.0%	|  1147 seconds
	Get games html 363: ==================================================  100.0%	|  1404 seconds
	Get games html 364: ==================================================  100.0%	|  1402 seconds
	Get games html 365: ==================================================  100.0%	|  2136 seconds
	Get games html 366: ==================================================  100.0%	|  2133 seconds
	Get games html 367: ==================================================  100.0%	|   628 seconds
	Get games html 368: ==================================================  100.0%	|  1386 seconds
	Get games html 369: ==================================================  100.0%	|  1094 seconds
	Get games html 370: ==================================================  100.0%	|  1185 seconds
	Get games html 371: ==================================================  100.0%	|  2196 seconds
	Get games html 372: ==================================================  100.0%	|  1435 seconds
	Get games html 373: ==================================================  100.0%	|  1077 seconds
	Get games html 374: ==================================================  100.0%	|  1202 seconds
	Get games html 375: ==================================================  100.0%	|  1408 seconds
	Get games html 376: ==================================================  100.0%	|  1079 seconds
	Get games html 377: ==================================================  100.0%	|  1363 seconds
	Get games html 378: ==================================================  100.0%	|  1053 seconds
	Get games html 379: ==================================================  100.0%	|  1051 seconds
	Get games html 380: ==================================================  100.0%	|   578 seconds
	Get games html 381: ==================================================  100.0%	|  1027 seconds
	Get games html 382: ==================================================  100.0%	|  1083 seconds
	Get games html 383: ==================================================  100.0%	|  1147 seconds
	Get games html 384: ==================================================  100.0%	|  1390 seconds
	Get games html 385: ==================================================  100.0%	|  1400 seconds
	Get games html 386: ==================================================  100.0%	|  2103 seconds
	Get games html 387: ==================================================  100.0%	|  1368 seconds
	Get games html 388: ==================================================  100.0%	|  1376 seconds
	Get games html 389: ==================================================  100.0%	|  2117 seconds
	Get games html 390: ==================================================  100.0%	|  1134 seconds
	Get games html 391: ==================================================  100.0%	|  1354 seconds
	Get games html 392: ==================================================  100.0%	|  1376 seconds
	Get games html 393: ==================================================  100.0%	|  2161 seconds
	Get games html 394: ==================================================  100.0%	|  2156 seconds
	Get games html 395: ==================================================  100.0%	|   598 seconds
	Get games html 396: ==================================================  100.0%	|  1369 seconds
	Get games html 397: ==================================================  100.0%	|  1110 seconds
	Get games html 398: ==================================================  100.0%	|  1196 seconds
	Get games html 399: ==================================================  100.0%	|  2146 seconds
	Get games html 400: ==================================================  100.0%	|  1396 seconds
	Get games html 401: ==================================================  100.0%	|  1051 seconds
	Get games html 402: ==================================================  100.0%	|  1220 seconds
	Get games html 403: ==================================================  100.0%	|  1384 seconds
	Get games html 404: ==================================================  100.0%	|  1061 seconds
	Get games html 405: ==================================================  100.0%	|  1382 seconds
	Get games html 406: ==================================================  100.0%	|  1104 seconds
	Get games html 407: ==================================================  100.0%	|  1066 seconds
	Get games html 408: ==================================================  100.0%	|   586 seconds
	Get games html 409: ==================================================  100.0%	|  1075 seconds
	Get games html 410: ==================================================  100.0%	|  1084 seconds
	Get games html 411: ==================================================  100.0%	|  1143 seconds
	Get games html 412: ==================================================  100.0%	|  1391 seconds
	Get games html 413: ==================================================  100.0%	|  1380 seconds
	Get games html 414: ==================================================  100.0%	|  1386 seconds
	Get games html 415: ==================================================  100.0%	|  1364 seconds
	Get games html 416: ==================================================  100.0%	|  1373 seconds
	Get games html 417: ==================================================  100.0%	|   565 seconds
	Get games html 418: ==================================================  100.0%	|   583 seconds
	Get games html 419: ==================================================  100.0%	|   570 seconds
	Get games html 420: ==================================================  100.0%	|   577 seconds
	Get games html 421: ==================================================  100.0%	|  1094 seconds
	Get games html 422: ==================================================  100.0%	|  1086 seconds
	Get games html 423: ==================================================  100.0%	|   599 seconds
	Get games html 424: ==================================================  100.0%	|  1068 seconds
	Get games html 425: ==================================================  100.0%	|  1573 seconds	
	Get games html 426: ==================================================  100.0%	|   572 seconds
	Get games html 427: ==================================================  100.0%	|  1063 seconds
	Get games html 428: ==================================================  100.0%	|  1105 seconds
	Get games html 429: ==================================================  100.0%	|  1156 seconds
	Get games html 430: ==================================================  100.0%	|  1179 seconds
	Get games html 431: ==================================================  100.0%	|  1067 seconds
	Get games html 432: ==================================================  100.0%	|   561 seconds
	Get games html 433: ==================================================  100.0%	|   579 seconds
	Get games html 434: ==================================================  100.0%	|  1138 seconds
	Get games html 435: ==================================================  100.0%	|  1386 seconds
	Get games html 436: ==================================================  100.0%	|  1331 seconds
	Get games html 437: ==================================================  100.0%	|  1340 seconds
	Get games html 438: ==================================================  100.0%	|  1111 seconds
	Get games html 439: ==================================================  100.0%	|  1361 seconds
	Get games html 440: ==================================================  100.0%	|  1383 seconds
	Get games html 441: ==================================================  100.0%	|   555 seconds
	Get games html 442: ==================================================  100.0%	|  1354 seconds
	Get games html 443: ==================================================  100.0%	|  1039 seconds
	Get games html 444: ==================================================  100.0%	|  1130 seconds
	Get games html 445: ==================================================  100.0%	|  1405 seconds
	Get games html 446: ==================================================  100.0%	|  1047 seconds
	Get games html 447: ==================================================  100.0%	|  1166 seconds
	Get games html 448: ==================================================  100.0%	|  1327 seconds
	Get games html 449: ==================================================  100.0%	|  1392 seconds
	Get games html 450: ==================================================  100.0%	|  1450 seconds
	Get games html 452: ==================================================  100.0%	|   652 seconds
	Get games html 453: ==================================================  100.0%	|  1140 seconds
	Get games html 454: ==================================================  100.0%	|  1160 seconds
	Get games html 455: ==================================================  100.0%	|  1218 seconds
	Get games html 456: ==================================================  100.0%	|  1475 seconds
	Get games html 457: ==================================================  100.0%	|   640 seconds
	Get games html 458: ==================================================  100.0%	|   674 seconds
	Get games html 459: ==================================================  100.0%	|  1225 seconds
	Get games html 460: ==================================================  100.0%	|  1163 seconds
	Get games html 461: ==================================================  100.0%	|  1150 seconds
	Get games html 462: ==================================================  100.0%	|  1457 seconds
	Get games html 463: ==================================================  100.0%	|  1436 seconds
	Get games html 464: ==================================================  100.0%	|  1430 seconds
	Get games html 465: ==================================================  100.0%	|  1425 seconds
	Get games html 466: ==================================================  100.0%	|  1459 seconds
	Get games html 467: ==================================================  100.0%	|   679 seconds
	Get games html 468: ==================================================  100.0%	|  1262 seconds
	Get games html 469: ==================================================  100.0%	|  1307 seconds
	Get games html 470: ==================================================  100.0%	|  1183 seconds
	Get games html 471: ==================================================  100.0%	|  1167 seconds
	Get games html 472: ==================================================  100.0%	|   703 seconds
	Get games html 473: ==================================================  100.0%	|  1183 seconds
	Get games html 474: ==================================================  100.0%	|  1208 seconds
	Get games html 475: ==================================================  100.0%	|  1251 seconds
	Get games html 476: ==================================================  100.0%	|  1511 seconds
	Get games html 477: ==================================================  100.0%	|   695 seconds
	Get games html 478: ==================================================  100.0%	|  2190 seconds
	Get games html 479: ==================================================  100.0%	|  2192 seconds
	Get games html 480: ==================================================  100.0%	|  1164 seconds
	Get games html 481: ==================================================  100.0%	|  2193 seconds
	Get games html 482: ==================================================  100.0%	|  1502 seconds
	Get games html 483: ==================================================  100.0%	|  1185 seconds
	Get games html 484: ==================================================  100.0%	|  1508 seconds
	Get games html 485: ==================================================  100.0%	|  1205 seconds
	Get games html 486: ==================================================  100.0%	|  1508 seconds
	Get games html 487: ==================================================  100.0%	|  1496 seconds
	Get games html 488: ==================================================  100.0%	|   694 seconds
	Get games html 488: ==================================================  100.0%	|  1436 seconds
	Get games html 489: ==================================================  100.0%	|  1409 seconds
	Get games html 490: ==================================================  100.0%	|  1483 seconds
	Get games html 491: ==================================================  100.0%	|  1265 seconds
	Get games html 492: ==================================================  100.0%	|  2207 seconds
	Get games html 493: ==================================================  100.0%	|  1308 seconds
	Get games html 494: ==================================================  100.0%	|  1475 seconds
	Get games html 495: ==================================================  100.0%	|  1482 seconds
	Get games html 496: ==================================================  100.0%	|   678 seconds
	Get games html 497: ==================================================  100.0%	|  1492 seconds
	Get games html 498: ==================================================  100.0%	|  1198 seconds
	Get games html 499: ==================================================  100.0%	|  1175 seconds
	Get games html 500: ==================================================  100.0%	|  1159 seconds
	Get games html 501: ==================================================  100.0%	|  1958 seconds	
	Get games html 504: ==================================================  100.0%	|  1014 seconds
	Get games html 505: ==================================================  100.0%	|  1943 seconds	
	Get games html 510: ==================================================  100.0%	|   928 seconds	
	Get games html 516: ==================================================  100.0%	|  1040 seconds	
	Get games html 545: ==================================================  100.0%	|   950 seconds
	Get games html 546: ==================================================  100.0%	|  1335 seconds	
	Get games html 559: ==================================================  100.0%	|  2002 seconds
	Get games html 560: ==================================================  100.0%	|  1079 seconds
	Get games html 570: ==================================================  100.0%	|   949 seconds	
	Get games html 570: ==================================================  100.0%	|   949 seconds	
	Get games html 576: ==================================================  100.0%	|  1354 seconds	
	Get games html 591: ==================================================  100.0%	|  1003 seconds	
	Get games html 606: ==================================================  100.0%	|  1566 seconds	
	Get games html 620: ==================================================  100.0%	|  1942 seconds	
	Get games html 621: ==================================================  100.0%	|  1009 seconds	
	Get games html 636: ==================================================  100.0%	|  1251 seconds	
	Get games html 650: ==================================================  100.0%	|   971 seconds
	Get games html 651: ==================================================  100.0%	|   736 seconds	
	Get games html 654: ==================================================  100.0%	|  1226 seconds	
	Get games html 666: ==================================================  100.0%	|  1586 seconds	
	Get games html 667: ==================================================  100.0%	|  1052 seconds	
	Get games html 681: ==================================================  100.0%	|  1228 seconds	
	Get games html 683: ==================================================  100.0%	|  1955 seconds
	Get games html 684: ==================================================  100.0%	|  1941 seconds
	Get games html 685: ==================================================  100.0%	|  1926 seconds
	Get games html 686: ==================================================  100.0%	|  1929 seconds
	Get games html 687: ==================================================  100.0%	|  1899 seconds
	Get games html 688: ==================================================  100.0%	|  1912 seconds
	Get games html 689: ==================================================  100.0%	|  1918 seconds
	Get games html 690: ==================================================  100.0%	|  1922 seconds
	Get games html 691: ==================================================  100.0%	|  1945 seconds
	Get games html 692: ==================================================  100.0%	|   942 seconds
	Get games html 693: ==================================================  100.0%	|  1002 seconds
	Get games html 694: ==================================================  100.0%	|  1948 seconds
	Get games html 695: ==================================================  100.0%	|  1008 seconds
	Get games html 696: ==================================================  100.0%	|  1298 seconds	
	Get games html 698: ==================================================  100.0%	|   433 seconds	
	Get games html 698: ==================================================  100.0%	|   433 seconds	
	Get games html 711: ==================================================  100.0%	|  2235 seconds	
	Get games html 716: ==================================================  100.0%	|  1907 seconds
	Get games html 717: ==================================================  100.0%	|  1913 seconds
	Get games html 718: ==================================================  100.0%	|  1895 seconds
	Get games html 719: ==================================================  100.0%	|  1884 seconds
	Get games html 720: ==================================================  100.0%	|  1957 seconds
	Get games html 721: ==================================================  100.0%	|  1968 seconds
	Get games html 722: ==================================================  100.0%	|  1983 seconds
	Get games html 723: ==================================================  100.0%	|  1963 seconds
	Get games html 724: ==================================================  100.0%	|  1981 seconds
	Get games html 725: ==================================================  100.0%	|  1101 seconds
	Get games html 726: ==================================================  100.0%	|  1587 seconds	
	Get games html 727: ==================================================  100.0%	|   472 seconds	
	Get games html 741: ==================================================  100.0%	|  1246 seconds	
	Get games html 743: ==================================================  100.0%	|   969 seconds	
	Get games html 756: ==================================================  100.0%	|  1351 seconds	
	Get games html 758: ==================================================  100.0%	|  1286 seconds	
	Get games html 771: ==================================================  100.0%	|  1638 seconds
	Get games html 772: ==================================================  100.0%	|  1083 seconds	
	Get games html 786: ==================================================  100.0%	|  1968 seconds
	Get games html 787: ==================================================  100.0%	|  1976 seconds	
	Get games html 792: ==================================================  100.0%	|   995 seconds	
	Get games html 797: ==================================================  100.0%	|  2044 seconds	
	Get games html 801: ==================================================  100.0%	|  1568 seconds	
	Get games html 803: ==================================================  100.0%	|   952 seconds	
	Get games html 816: ==================================================  100.0%	|  1583 seconds	
	Get games html 830: ==================================================  100.0%	|  1024 seconds	
	Get games html 831: ==================================================  100.0%	|  1348 seconds	
	Get games html 838: ==================================================  100.0%	|   972 seconds	
	Get games html 846: ==================================================  100.0%	|  1287 seconds	
	Get games html 849: ==================================================  100.0%	|  1252 seconds	
	Get games html 861: ==================================================  100.0%	|   762 seconds	
	Get games html 868: ==================================================  100.0%	|   980 seconds	
	Get games html 876: ==================================================  100.0%	|  1238 seconds	
	Get games html 878: ==================================================  100.0%	|   939 seconds	
	Get games html 885: ==================================================  100.0%	|   979 seconds	
	Get games html 891: ==================================================  100.0%	|  1312 seconds	
	Get games html 893: ==================================================  100.0%	|   983 seconds	
	Get games html 900: ==================================================  100.0%	|   957 seconds	
	Get games html 906: ==================================================  100.0%	|  1348 seconds	
	Get games html 907: ==================================================  100.0%	|  1239 seconds	
	Get games html 910: ==================================================  100.0%	|  1208 seconds	
	Get games html 920: ==================================================  100.0%	|  1062 seconds
	Get games html 922: ==================================================  100.0%	|   990 seconds	
	Get games html 936:==================================================  100.0%	|  1022 seconds
	Get games html 946: ==================================================  100.0%	|   516 seconds
	Get games html 948: ==================================================  100.0%	|   430 seconds
	Get games html 949: ==================================================  100.0%	|   516 seconds
	Get games html 951: ==================================================  100.0%	|  1202 seconds
	Get games html 954: ==================================================  100.0%	|   976 seconds
    Get games html 006: ==================================================  100.0%	|  2131 seconds
    Get games html 451: ==================================================  100.0%	|  1145 seconds
    Get games html 502: ==================================================  100.0%	|  1955 seconds	
    Get games html 503: ==================================================  100.0%	|  1952 seconds	
    Get games html 506: ==================================================  100.0%	|  1929 seconds	
    Get games html 507: ==================================================  100.0%	|  1897 seconds	
    Get games html 508: ==================================================  100.0%	|  1949 seconds	
    Get games html 509: ==================================================  100.0%	|  1953 seconds
    Get games html 511: ==================================================  100.0%	|   931 seconds	
    Get games html 512: ==================================================  100.0%	|   915 seconds	
    Get games html 513: ==================================================  100.0%	|   946 seconds	
    Get games html 514: ==================================================  100.0%	|   962 seconds	
    Get games html 515: ==================================================  100.0%	|   956 seconds
    Get games html 517: ==================================================  100.0%	|  1036 seconds	
    Get games html 518: ==================================================  100.0%	|  1016 seconds	
    Get games html 519: ==================================================  100.0%	|  1007 seconds	
    Get games html 520: ==================================================  100.0%	|   997 seconds	
    Get games html 521: ==================================================  100.0%	|   985 seconds	
    Get games html 522: ==================================================  100.0%	|   995 seconds	
    Get games html 523: ==================================================  100.0%	|   987 seconds	
    Get games html 524: ==================================================  100.0%	|   981 seconds	
    Get games html 525: ==================================================  100.0%	|  1044 seconds	
    Get games html 526: ==================================================  100.0%	|  1030 seconds	
    Get games html 527: ==================================================  100.0%	|  1036 seconds	
    Get games html 528: ==================================================  100.0%	|  1012 seconds	
    Get games html 529: ==================================================  100.0%	|  1038 seconds	
    Get games html 530: ==================================================  100.0%	|  1030 seconds
    Get games html 531: ==================================================  100.0%	|  1522 seconds	
    Get games html 532: ==================================================  100.0%	|  1608 seconds	
    Get games html 533: ==================================================  100.0%	|  1702 seconds	
    Get games html 534: ==================================================  100.0%	|  1795 seconds	
    Get games html 535: ==================================================  100.0%	|  1488 seconds	
    Get games html 536: ==================================================  100.0%	|  1402 seconds	
    Get games html 537: ==================================================  100.0%	|  1356 seconds	
    Get games html 538: ==================================================  100.0%	|  1577 seconds	
    Get games html 539: ==================================================  100.0%	|  1295 seconds	
    Get games html 540: ==================================================  100.0%	|  1271 seconds	
    Get games html 541: ==================================================  100.0%	|  1243 seconds	
    Get games html 542: ==================================================  100.0%	|  1239 seconds	
    Get games html 543: ==================================================  100.0%	|  1237 seconds	
    Get games html 544: ==================================================  100.0%	|  1223 seconds
    Get games html 547: ==================================================  100.0%	|  1380 seconds	
    Get games html 548: ==================================================  100.0%	|  1365 seconds	
    Get games html 549: ==================================================  100.0%	|  1353 seconds	
    Get games html 550: ==================================================  100.0%	|  1326 seconds	
    Get games html 551: ==================================================  100.0%	|  1310 seconds	
    Get games html 552: ==================================================  100.0%	|  1257 seconds	
    Get games html 553: ==================================================  100.0%	|  1283 seconds	
    Get games html 554: ==================================================  100.0%	|  1309 seconds	
    Get games html 555: ==================================================  100.0%	|  1391 seconds	
    Get games html 556: ==================================================  100.0%	|  1399 seconds	
    Get games html 557: ==================================================  100.0%	|  1397 seconds	
    Get games html 558: ==================================================  100.0%	|  1420 seconds
    Get games html 561: ==================================================  100.0%	|  2315 seconds	
    Get games html 562: ==================================================  100.0%	|  2474 seconds	
    Get games html 563: ==================================================  100.0%	|  2493 seconds	
    Get games html 564: ==================================================  100.0%	|  2207 seconds	
    Get games html 565: ==================================================  100.0%	|  2221 seconds	
    Get games html 566: ==================================================  100.0%	|  2163 seconds	
    Get games html 567: ==================================================  100.0%	|  2041 seconds	
    Get games html 568: ==================================================  100.0%	|  2017 seconds	
    Get games html 569: ==================================================  100.0%	|  1994 seconds
    Get games html 571: ==================================================  100.0%	|   943 seconds	
    Get games html 571: ==================================================  100.0%	|   943 seconds	
    Get games html 572: ==================================================  100.0%	|   949 seconds	
    Get games html 572: ==================================================  100.0%	|   949 seconds	
    Get games html 573: ==================================================  100.0%	|   960 seconds	
    Get games html 573: ==================================================  100.0%	|   960 seconds	
    Get games html 574: ==================================================  100.0%	|   953 seconds	
    Get games html 574: ==================================================  100.0%	|   953 seconds	
    Get games html 575: ==================================================  100.0%	|   947 seconds
    Get games html 575: ==================================================  100.0%	|   947 seconds
    Get games html 577: ==================================================  100.0%	|  1381 seconds	
    Get games html 578: ==================================================  100.0%	|  1485 seconds	
    Get games html 579: ==================================================  100.0%	|  1586 seconds	
    Get games html 580: ==================================================  100.0%	|  1420 seconds	
    Get games html 581: ==================================================  100.0%	|  1255 seconds	
    Get games html 582: ==================================================  100.0%	|  1179 seconds	
    Get games html 583: ==================================================  100.0%	|  1178 seconds	
    Get games html 584: ==================================================  100.0%	|  1428 seconds	
    Get games html 585: ==================================================  100.0%	|  1093 seconds	
    Get games html 586: ==================================================  100.0%	|  1068 seconds	
    Get games html 587: ==================================================  100.0%	|  1064 seconds	
    Get games html 588: ==================================================  100.0%	|  1010 seconds	
    Get games html 589: ==================================================  100.0%	|  1024 seconds	
    Get games html 590: ==================================================  100.0%	|  1022 seconds
    Get games html 592: ==================================================  100.0%	|   987 seconds	
    Get games html 593: ==================================================  100.0%	|   987 seconds	
    Get games html 594: ==================================================  100.0%	|   982 seconds	
    Get games html 595: ==================================================  100.0%	|  1000 seconds	
    Get games html 596: ==================================================  100.0%	|   987 seconds	
    Get games html 597: ==================================================  100.0%	|  1010 seconds	
    Get games html 598: ==================================================  100.0%	|   987 seconds	
    Get games html 599: ==================================================  100.0%	|   976 seconds	
    Get games html 600: ==================================================  100.0%	|  1009 seconds	
    Get games html 601: ==================================================  100.0%	|  1000 seconds	
    Get games html 602: ==================================================  100.0%	|  1006 seconds	
    Get games html 603: ==================================================  100.0%	|  1021 seconds	
    Get games html 604: ==================================================  100.0%	|  1020 seconds	
    Get games html 605: ==================================================  100.0%	|  1046 seconds
    Get games html 607: ==================================================  100.0%	|  1573 seconds	
    Get games html 608: ==================================================  100.0%	|  1690 seconds	
    Get games html 609: ==================================================  100.0%	|  1785 seconds	
    Get games html 610: ==================================================  100.0%	|  1438 seconds	
    Get games html 611: ==================================================  100.0%	|  1404 seconds	
    Get games html 612: ==================================================  100.0%	|  1431 seconds	
    Get games html 613: ==================================================  100.0%	|  1659 seconds	
    Get games html 614: ==================================================  100.0%	|  1339 seconds	
    Get games html 615: ==================================================  100.0%	|  1340 seconds	
    Get games html 616: ==================================================  100.0%	|  1305 seconds	
    Get games html 617: ==================================================  100.0%	|  1299 seconds	
    Get games html 618: ==================================================  100.0%	|  1259 seconds	
    Get games html 619: ==================================================  100.0%	|  1254 seconds	
    Get games html 622: ==================================================  100.0%	|   986 seconds	
    Get games html 623: ==================================================  100.0%	|   998 seconds	
    Get games html 624: ==================================================  100.0%	|   970 seconds	
    Get games html 625: ==================================================  100.0%	|   987 seconds	
    Get games html 626: ==================================================  100.0%	|   986 seconds	
    Get games html 627: ==================================================  100.0%	|   980 seconds	
    Get games html 628: ==================================================  100.0%	|   955 seconds	
    Get games html 629: ==================================================  100.0%	|   956 seconds	
    Get games html 630: ==================================================  100.0%	|   956 seconds	
    Get games html 631: ==================================================  100.0%	|   994 seconds	
    Get games html 632: ==================================================  100.0%	|   994 seconds	
    Get games html 633: ==================================================  100.0%	|   974 seconds	
    Get games html 634: ==================================================  100.0%	|  1012 seconds	
    Get games html 635: ==================================================  100.0%	|   993 seconds	
    Get games html 637: ==================================================  100.0%	|  1236 seconds	
    Get games html 638: ==================================================  100.0%	|  1227 seconds	
    Get games html 639: ==================================================  100.0%	|  1210 seconds	
    Get games html 640: ==================================================  100.0%	|  1226 seconds	
    Get games html 641: ==================================================  100.0%	|  1258 seconds	
    Get games html 642: ==================================================  100.0%	|  1243 seconds	
    Get games html 643: ==================================================  100.0%	|  1244 seconds	
    Get games html 644: ==================================================  100.0%	|  1233 seconds	
    Get games html 645: ==================================================  100.0%	|  1265 seconds	
    Get games html 646: ==================================================  100.0%	|  1225 seconds	
    Get games html 647: ==================================================  100.0%	|  1232 seconds	
    Get games html 648: ==================================================  100.0%	|  1267 seconds	
    Get games html 649: ==================================================  100.0%	|  1277 seconds
    Get games html 652: ==================================================  100.0%	|   792 seconds	
    Get games html 653: ==================================================  100.0%	|   763 seconds
    Get games html 655: ==================================================  100.0%	|  1199 seconds	
    Get games html 656: ==================================================  100.0%	|  1192 seconds	
    Get games html 657: ==================================================  100.0%	|  1182 seconds	
    Get games html 658: ==================================================  100.0%	|  1182 seconds	
    Get games html 659: ==================================================  100.0%	|  1184 seconds	
    Get games html 660: ==================================================  100.0%	|  1167 seconds	
    Get games html 661: ==================================================  100.0%	|  1201 seconds	
    Get games html 662: ==================================================  100.0%	|  1216 seconds	
    Get games html 663: ==================================================  100.0%	|  1203 seconds	
    Get games html 664: ==================================================  100.0%	|  1236 seconds	
    Get games html 665: ==================================================  100.0%	|  1231 seconds	
    Get games html 668: ==================================================  100.0%	|  1026 seconds	
    Get games html 669: ==================================================  100.0%	|   999 seconds	
    Get games html 670: ==================================================  100.0%	|  1012 seconds	
    Get games html 671: ==================================================  100.0%	|  1002 seconds	
    Get games html 672: ==================================================  100.0%	|   983 seconds	
    Get games html 673: ==================================================  100.0%	|   990 seconds	
    Get games html 674: ==================================================  100.0%	|   990 seconds	
    Get games html 675: ==================================================  100.0%	|   980 seconds	
    Get games html 676: ==================================================  100.0%	|  1002 seconds	
    Get games html 677: ==================================================  100.0%	|   988 seconds	
    Get games html 678: ==================================================  100.0%	|  1005 seconds	
    Get games html 679: ==================================================  100.0%	|  1006 seconds	
    Get games html 680: ==================================================  100.0%	|  1027 seconds	
    Get games html 682: ==================================================  100.0%	|  1237 seconds
    Get games html 697: ==================================================  100.0%	|  1303 seconds	
    Get games html 699: ==================================================  100.0%	|   424 seconds	
    Get games html 699: ==================================================  100.0%	|   424 seconds	
    Get games html 700: ==================================================  100.0%	|   436 seconds	
    Get games html 700: ==================================================  100.0%	|   436 seconds	
    Get games html 701: ==================================================  100.0%	|   439 seconds	
    Get games html 701: ==================================================  100.0%	|   439 seconds	
    Get games html 702: ==================================================  100.0%	|   413 seconds	
    Get games html 702: ==================================================  100.0%	|   413 seconds	
    Get games html 703: ==================================================  100.0%	|   425 seconds	
    Get games html 703: ==================================================  100.0%	|   425 seconds	
    Get games html 704: ==================================================  100.0%	|   448 seconds	
    Get games html 704: ==================================================  100.0%	|   448 seconds	
    Get games html 705: ==================================================  100.0%	|   426 seconds	
    Get games html 705: ==================================================  100.0%	|   426 seconds	
    Get games html 706: ==================================================  100.0%	|   418 seconds	
    Get games html 706: ==================================================  100.0%	|   418 seconds	
    Get games html 707: ==================================================  100.0%	|   414 seconds	
    Get games html 707: ==================================================  100.0%	|   414 seconds	
    Get games html 708: ==================================================  100.0%	|   433 seconds	
    Get games html 708: ==================================================  100.0%	|   433 seconds	
    Get games html 709: ==================================================  100.0%	|   420 seconds	
    Get games html 709: ==================================================  100.0%	|   420 seconds	
    Get games html 710: ==================================================  100.0%	|   421 seconds	
    Get games html 710: ==================================================  100.0%	|   421 seconds	
    Get games html 712: ==================================================  100.0%	|  2380 seconds	
    Get games html 713: ==================================================  100.0%	|  2383 seconds	
    Get games html 714: ==================================================  100.0%	|  2086 seconds	
    Get games html 715: ==================================================  100.0%	|  2027 seconds	
    Get games html 728: ==================================================  100.0%	|   454 seconds	
    Get games html 729: ==================================================  100.0%	|   455 seconds	
    Get games html 730: ==================================================  100.0%	|   464 seconds	
    Get games html 731: ==================================================  100.0%	|   453 seconds	
    Get games html 732: ==================================================  100.0%	|   454 seconds	
    Get games html 733: ==================================================  100.0%	|   452 seconds	
    Get games html 734: ==================================================  100.0%	|   444 seconds	
    Get games html 735: ==================================================  100.0%	|   426 seconds	
    Get games html 736: ==================================================  100.0%	|   454 seconds	
    Get games html 737: ==================================================  100.0%	|   440 seconds	
    Get games html 738: ==================================================  100.0%	|   428 seconds	
    Get games html 739: ==================================================  100.0%	|   436 seconds	
    Get games html 740: ==================================================  100.0%	|   448 seconds
    Get games html 742: ==================================================  100.0%	|  1244 seconds	
    Get games html 744: ==================================================  100.0%	|   957 seconds	
    Get games html 745: ==================================================  100.0%	|   958 seconds	
    Get games html 746: ==================================================  100.0%	|   943 seconds	
    Get games html 747: ==================================================  100.0%	|   962 seconds	
    Get games html 748: ==================================================  100.0%	|   958 seconds	
    Get games html 749: ==================================================  100.0%	|   961 seconds	
    Get games html 750: ==================================================  100.0%	|   953 seconds	
    Get games html 751: ==================================================  100.0%	|   941 seconds	
    Get games html 752: ==================================================  100.0%	|   951 seconds	
    Get games html 753: ==================================================  100.0%	|   971 seconds	
    Get games html 754: ==================================================  100.0%	|  1069 seconds	
    Get games html 755: ==================================================  100.0%	|  1065 seconds	
    Get games html 757: ==================================================  100.0%	|  1586 seconds	
    Get games html 759: ==================================================  100.0%	|  1316 seconds	
    Get games html 760: ==================================================  100.0%	|  1264 seconds	
    Get games html 761: ==================================================  100.0%	|  1152 seconds	
    Get games html 762: ==================================================  100.0%	|  1239 seconds	
    Get games html 763: ==================================================  100.0%	|  1304 seconds	
    Get games html 764: ==================================================  100.0%	|  1231 seconds	
    Get games html 765: ==================================================  100.0%	|  1306 seconds	
    Get games html 766: ==================================================  100.0%	|  1330 seconds	
    Get games html 767: ==================================================  100.0%	|  1305 seconds	
    Get games html 768: ==================================================  100.0%	|  1263 seconds	
    Get games html 769: ==================================================  100.0%	|  1359 seconds	
    Get games html 770: ==================================================  100.0%	|  1331 seconds
    Get games html 773: ==================================================  100.0%	|  1050 seconds	
    Get games html 774: ==================================================  100.0%	|   993 seconds	
    Get games html 775: ==================================================  100.0%	|  1086 seconds	
    Get games html 776: ==================================================  100.0%	|  1088 seconds	
    Get games html 777: ==================================================  100.0%	|  1009 seconds	
    Get games html 778: ==================================================  100.0%	|   987 seconds	
    Get games html 779: ==================================================  100.0%	|   997 seconds	
    Get games html 780: ==================================================  100.0%	|  1028 seconds	
    Get games html 781: ==================================================  100.0%	|   941 seconds	
    Get games html 782: ==================================================  100.0%	|   996 seconds	
    Get games html 783: ==================================================  100.0%	|  1004 seconds	
    Get games html 784: ==================================================  100.0%	|  1011 seconds	
    Get games html 785: ==================================================  100.0%	|  1040 seconds	
    Get games html 788: ==================================================  100.0%	|  1960 seconds	
    Get games html 789: ==================================================  100.0%	|  1927 seconds	
    Get games html 790: ==================================================  100.0%	|  1990 seconds	
    Get games html 791: ==================================================  100.0%	|  1983 seconds
    Get games html 793: ==================================================  100.0%	|   971 seconds	
    Get games html 794: ==================================================  100.0%	|   970 seconds	
    Get games html 795: ==================================================  100.0%	|  1008 seconds	
    Get games html 796: ==================================================  100.0%	|  1049 seconds
    Get games html 798: ==================================================  100.0%	|  1909 seconds	
    Get games html 799: ==================================================  100.0%	|  1912 seconds	
    Get games html 800: ==================================================  100.0%	|  1958 seconds	
    Get games html 802: ==================================================  100.0%	|  1583 seconds
    Get games html 804: ==================================================  100.0%	|   929 seconds	
    Get games html 805: ==================================================  100.0%	|   960 seconds	
    Get games html 806: ==================================================  100.0%	|   931 seconds	
    Get games html 807: ==================================================  100.0%	|   933 seconds	
    Get games html 808: ==================================================  100.0%	|   939 seconds	
    Get games html 809: ==================================================  100.0%	|   956 seconds	
    Get games html 810: ==================================================  100.0%	|   937 seconds	
    Get games html 811: ==================================================  100.0%	|  1024 seconds	
    Get games html 812: ==================================================  100.0%	|   977 seconds	
    Get games html 813: ==================================================  100.0%	|  1061 seconds	
    Get games html 814: ==================================================  100.0%	|   982 seconds	
    Get games html 815: ==================================================  100.0%	|   944 seconds	
    Get games html 817: ==================================================  100.0%	|  1713 seconds	
    Get games html 818: ==================================================  100.0%	|  1840 seconds	
    Get games html 819: ==================================================  100.0%	|  1959 seconds	
    Get games html 820: ==================================================  100.0%	|  1721 seconds	
    Get games html 821: ==================================================  100.0%	|  1646 seconds	
    Get games html 822: ==================================================  100.0%	|  1744 seconds	
    Get games html 823: ==================================================  100.0%	|  1246 seconds	
    Get games html 824: ==================================================  100.0%	|  1306 seconds	
    Get games html 825: ==================================================  100.0%	|  1272 seconds	
    Get games html 826: ==================================================  100.0%	|  1285 seconds	
    Get games html 827: ==================================================  100.0%	|  1281 seconds	
    Get games html 828: ==================================================  100.0%	|  1258 seconds	
    Get games html 829: ==================================================  100.0%	|  1254 seconds	
    Get games html 832: ==================================================  100.0%	|  1374 seconds	
    Get games html 833: ==================================================  100.0%	|  1442 seconds	
    Get games html 834: ==================================================  100.0%	|  1504 seconds	
    Get games html 835: ==================================================  100.0%	|  1368 seconds	
    Get games html 836: ==================================================  100.0%	|  1159 seconds	
    Get games html 837: ==================================================  100.0%	|  1133 seconds	
    Get games html 839: ==================================================  100.0%	|   977 seconds	
    Get games html 840: ==================================================  100.0%	|  1009 seconds	
    Get games html 841: ==================================================  100.0%	|   988 seconds	
    Get games html 842: ==================================================  100.0%	|   998 seconds	
    Get games html 843: ==================================================  100.0%	|   977 seconds	
    Get games html 844: ==================================================  100.0%	|   989 seconds	
    Get games html 845: ==================================================  100.0%	|   998 seconds	
    Get games html 847: ==================================================  100.0%	|  1286 seconds	
    Get games html 848: ==================================================  100.0%	|  1429 seconds	
    Get games html 850: ==================================================  100.0%	|  1247 seconds	
    Get games html 851: ==================================================  100.0%	|  1214 seconds	
    Get games html 852: ==================================================  100.0%	|  1235 seconds	
    Get games html 853: ==================================================  100.0%	|  1152 seconds	
    Get games html 854: ==================================================  100.0%	|  1198 seconds	
    Get games html 855: ==================================================  100.0%	|  1214 seconds	
    Get games html 856: ==================================================  100.0%	|  1294 seconds	
    Get games html 857: ==================================================  100.0%	|  1296 seconds	
    Get games html 858: ==================================================  100.0%	|  1283 seconds	
    Get games html 859: ==================================================  100.0%	|  1302 seconds	
    Get games html 860: ==================================================  100.0%	|  1303 seconds	
    Get games html 862: ==================================================  100.0%	|   742 seconds	
    Get games html 863: ==================================================  100.0%	|   733 seconds	
    Get games html 864: ==================================================  100.0%	|   892 seconds	
    Get games html 865: ==================================================  100.0%	|   912 seconds	
    Get games html 866: ==================================================  100.0%	|  1033 seconds	
    Get games html 867: ==================================================  100.0%	|   975 seconds	
    Get games html 869: ==================================================  100.0%	|   946 seconds	
    Get games html 870: ==================================================  100.0%	|   975 seconds	 
    Get games html 871: ==================================================  100.0%	|   977 seconds	
    Get games html 872: ==================================================  100.0%	|   919 seconds	
    Get games html 873: ==================================================  100.0%	|   945 seconds	
    Get games html 874: ==================================================  100.0%	|   956 seconds	
    Get games html 875: ==================================================  100.0%	|   957 seconds	
    Get games html 877: ==================================================  100.0%	|  1226 seconds
    Get games html 879: ==================================================  100.0%	|  1034 seconds	
    Get games html 880: ==================================================  100.0%	|   979 seconds	
    Get games html 881: ==================================================  100.0%	|   955 seconds	
    Get games html 882: ==================================================  100.0%	|   957 seconds	
    Get games html 883: ==================================================  100.0%	|  1010 seconds	
    Get games html 884: ==================================================  100.0%	|  1017 seconds
    Get games html 886: ==================================================  100.0%	|   971 seconds	
    Get games html 887: ==================================================  100.0%	|   952 seconds	
    Get games html 888: ==================================================  100.0%	|   959 seconds	
    Get games html 889: ==================================================  100.0%	|   980 seconds	
    Get games html 890: ==================================================  100.0%	|   967 seconds	
    Get games html 892: ==================================================  100.0%	|  1297 seconds	
    Get games html 894: ==================================================  100.0%	|   974 seconds	
    Get games html 895: ==================================================  100.0%	|  1013 seconds	
    Get games html 896: ==================================================  100.0%	|  1002 seconds	
    Get games html 897: ==================================================  100.0%	|  1000 seconds	
    Get games html 898: ==================================================  100.0%	|  1014 seconds	
    Get games html 899: ==================================================  100.0%	|   977 seconds
    Get games html 901: ==================================================  100.0%	|   949 seconds	
    Get games html 902: ==================================================  100.0%	|   944 seconds	
    Get games html 903: ==================================================  100.0%	|   953 seconds	
    Get games html 904: ==================================================  100.0%	|   939 seconds	
    Get games html 905: ==================================================  100.0%	|   952 seconds	
    Get games html 908: ==================================================  100.0%	|  1213 seconds	
    Get games html 909: ==================================================  100.0%	|  1213 seconds
    Get games html 911: ==================================================  100.0%	|  1221 seconds	
    Get games html 912: ==================================================  100.0%	|  1204 seconds	
    Get games html 913: ==================================================  100.0%	|  1250 seconds	
    Get games html 914: ==================================================  100.0%	|  1273 seconds	
    Get games html 915: ==================================================  100.0%	|  1271 seconds	
    Get games html 916: ==================================================  100.0%	|  1248 seconds	
    Get games html 917: ==================================================  100.0%	|  1252 seconds	
    Get games html 918: ==================================================  100.0%	|  1264 seconds	
    Get games html 919: ==================================================  100.0%	|  1280 seconds
    Get games html 921: ==================================================  100.0%	|  1727 seconds
    Get games html 923: ==================================================  100.0%	|  1055 seconds	
    Get games html 924: ==================================================  100.0%	|  1028 seconds	
    Get games html 925: ==================================================  100.0%	|   995 seconds	
    Get games html 926: ==================================================  100.0%	|   964 seconds	
    Get games html 927: ==================================================  100.0%	|   979 seconds	
    Get games html 928: ==================================================  100.0%	|   971 seconds	
    Get games html 929: ==================================================  100.0%	|   989 seconds	
    Get games html 930: ==================================================  100.0%	|  1003 seconds	
    Get games html 931: ==================================================  100.0%	|   971 seconds	
    Get games html 932: ==================================================  100.0%	|  1039 seconds	
    Get games html 933: ==================================================  100.0%	|   961 seconds	
    Get games html 934: ==================================================  100.0%	|  1021 seconds	
    Get games html 935: ==================================================  100.0%	|  1027 seconds
    Get games html 937: ==================================================  100.0%	|   869 seconds
    Get games html 938: ==================================================  100.0%	|   922 seconds
    Get games html 939: ==================================================  100.0%	|   979 seconds
    Get games html 940: ==================================================  100.0%	|  1307 seconds
    Get games html 941: ==================================================  100.0%	|   978 seconds
    Get games html 942: ==================================================  100.0%	|   998 seconds
    Get games html 943: ==================================================  100.0%	|   954 seconds
    Get games html 944: ==================================================  100.0%	|  1946 seconds
    Get games html 945: ==================================================  100.0%	|  1005 seconds
    Get games html 947: ==================================================  100.0%	|   974 seconds
    Get games html 950: ==================================================  100.0%	|   416 seconds
    Get games html 950: ==================================================  100.0%	|   416 seconds
    Get games html 952: ==================================================  100.0%	|  1019 seconds
    Get games html 953: ==================================================  100.0%	|  1234 seconds
    Get games html 955: ==================================================  100.0%	|  1023 seconds
    Get games html 956: ==================================================  100.0%	|  1191 seconds
Finished in 1259700 seconds
```

##  Squads

In [ ]:
squads_html = pd.read_csv("./drive/MyDrive/transfermarkt/3-games/games_html_0.csv")
squads_html["url"] = list(map(lambda x: x.replace("index", "aufstellung"), squads_html["url"]))
del squads_html["campaign_id"]
squads_html.columns = ["game_id","url","format","content"]
squads_html.head()
squads_html.to_csv("./drive/MyDrive/transfermarkt/6-squads/squads_html_0.csv", index=False)

In [ ]:
squads_html.shape

```
(963104, 4)
```

```
Finished in 1259700 seconds
```

##### Very long result

## Games preview - New

In [ ]:
games_preview_html = pd.DataFrame(columns=["id","campaign_id","url","format","content"])

In [ ]:
def get_games_preview_url(ind):
    global campaigns_html
    campaign = campaigns_html.iloc[ind]
    if "ERROR 500" in campaign["content"]:
        return []
    url = campaign["url"]
    campaign_id = campaign["id"]
    content = campaign["content"]
    format = campaign["format"]
    soup = BeautifulSoup(content, "lxml")
    tags = soup.find_all('a', {'title': 'Match preview'}) + soup.find_all('a', {'title': 'To the preview'})
    df = []
    for tag in tags:
        df.append([campaign_id + "_" + tag.get('href').split('/')[-1], campaign_id, base_url + tag.get('href'), format])
    return df

In [ ]:
# June 11
print("Get games preview url: ", end="")
start_time = time.time()
nv = 12
for v in range(1,nv+1):
    campaigns_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/2-campaigns/campaigns_html_{v:03}.csv")
    p = Pool(20)
    games_preview_html = games_preview_html.append(pd.DataFrame(reduce(lambda a, b: a + b, \
                        p.map(get_games_preview_url, range(campaigns_html.shape[0]))), \
                        columns=["id","campaign_id","url","format"]), \
                        ignore_index=True)
    p.terminate()
    p.join()
    p = 100*v/nv
    print(f"\r{'='*round(p/2)}{' '*(50-round(p/2))} {p:.2f}%", end="")
games_preview_html.to_csv("./drive/MyDrive/transfermarkt/3-games/games_preview_html_0.csv", index=False)
print(f"{round(time.time() - start_time)} seconds")

```
Get games preview url: 
================================================== 100.00%
1486 seconds
```

In [ ]:
games_preview_html.shape

(339078, 5)

In [ ]:
get_html("games preview", games_html, "./drive/MyDrive/transfermarkt/3-games/games_preview_html")

```

```

## Teams

In [ ]:
teams_html = pd.DataFrame(columns=["id","name","url","content"])

In [ ]:
def get_teams_url(ind):
    global games_html
    game = games_html.iloc[ind]
    content = game["content"]
    if "ERROR 500" in content:
        return []
    soup = BeautifulSoup(content, "lxml")
    info = soup.find("div", {"class": "box-content"})
    df = []
    for _class in ["sb-heim", "sb-gast"]:
        div = info.find('div', {'class': _class})
        id = div.find('a').get('id')
        name = div.text.strip()
        url = base_url + div.find('a').get('href')[:-15]
        df.append([id, name, url])
    return df

In [ ]:
print("Get teams url: ")
start_time = time.time()
nv = 956
for v in range(1,nv+1):
    games_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/3-games/games_html_{v:03}.csv")
    p = Pool(20)
    teams_html = teams_html.append(pd.DataFrame(reduce(lambda a, b: a + b, p.map(get_teams_url, range(games_html.shape[0]))), \
                                                columns=["id","name","url"]), ignore_index=True)
    teams_html = teams_html.drop_duplicates(subset="url").reset_index(drop=True)
    p.terminate()
    p.join()
    p = 100*v/nv
    print(f"\r{'='*round(p/2)}{' '*(50-round(p/2))} {p:.2f}%", end="")
teams_html.to_csv("./drive/MyDrive/transfermarkt/7-teams/teams_html_0.csv", index=False)
print(f"\nFinished in {round(time.time() - start_time)} seconds")

In [ ]:
print("Get teams url: ")
start_time = time.time()
for v in range(956,965):
    games_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/3-games/games_html_{v:03}.csv")
    p = Pool(20)
    teams_html = teams_html.append(pd.DataFrame(reduce(lambda a, b: a + b, p.map(get_teams_url, range(games_html.shape[0]))), \
                                                columns=["id","name","url"]), ignore_index=True)
    teams_html = teams_html.drop_duplicates(subset="url").reset_index(drop=True)
    p.terminate()
    p.join()
    print(v)
teams_html.to_csv("./drive/MyDrive/transfermarkt/7-teams/teams_html_0.csv", index=False)
print(f"\nFinished in {round(time.time() - start_time)} seconds")

```
Get teams url: 
================================================== 100.00%
Finished in 36047 seconds
```

In [ ]:
teams_html["name"] = [re.sub(" Position:.*", "", x) for x in teams_html["name"]]
teams_html.to_csv("./drive/MyDrive/transfermarkt/7-teams/teams_html_0.csv", index=False)

In [ ]:
print("Get teams html: ")
start_time = time.time()
nv = 17
for v in range(1,nv+1):
    teams_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/7-teams/teams_html_0.csv")
    get_html_large("teams", teams_html, "./drive/MyDrive/transfermarkt/7-teams/teams_html", v)
print(f"Finished in {round(time.time()-start_time)} seconds")

```
Get teams html:
    Get teams html 001: ==================================================  100.0%	|  2720 seconds 
	Get teams html 002: ==================================================  100.0%	|  2418 seconds
	Get teams html 003: ==================================================  100.0%	|  2277 seconds
	Get teams html 004: ==================================================  100.0%	|  2436 seconds
	Get teams html 005: ==================================================  100.0%	|  2434 seconds
	Get teams html 006: ==================================================  100.0%	|  2339 seconds
	Get teams html 007: ==================================================  100.0%	|  2397 seconds
	Get teams html 008: ==================================================  100.0%	|  2009 seconds 
	Get teams html 009: ==================================================  100.0%	|  2134 seconds
    Get teams html 010: ==================================================  100.0%	|  2418 seconds
    Get teams html 011: ==================================================  100.0%	|  2104 seconds
    Get teams html 012: ==================================================  100.0%	|  2156 seconds
	Get teams html 013: ==================================================  100.0%	|  2248 seconds
	Get teams html 014: ==================================================  100.0%	|  2401 seconds
	Get teams html 015: ==================================================  100.0%	|  2432 seconds
	Get teams html 016: ==================================================  100.0%	|  1456 seconds
Finished in 36379 seconds
```

## Players

## Stadiums - New

In [ ]:
stadiums_html = pd.DataFrame(columns=["id","team_id","stadium_id","name","url","content"])

In [ ]:
team_stadiums_html = pd.read_csv("./drive/MyDrive/transfermarkt/7-teams/teams_html_0.csv")
team_stadiums_html["url"] = list(map(lambda x: x.replace("startseite", "stadion"), team_stadiums_html["url"]))
team_stadiums_html.to_csv("./drive/MyDrive/transfermarkt/9-stadiums/team_stadiums_html_0.csv", index=False)

In [ ]:
team_stadiums_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/9-stadiums/team_stadiums_html_0.csv")
for i in range()

In [ ]:
print("Get team stadiums html: ")
start_time = time.time()
for v in range(17):
    team_stadiums_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/9-stadiums/team_stadiums_html_0.csv")
    get_html("teams", teams_html, "./drive/MyDrive/transfermarkt/9-stadiums/team stadiums_data", v)
print(f"Finished in {round(time.time()-start_time)} seconds")

Get team stadiums html: 
	Get teams html 957: 

	Get teams html 958: 

	Get teams html 959: 

	Get teams html 960: 

	Get teams html 961: 

	Get teams html 962: 

	Get teams html 963: 

	Get teams html 964: 

Finished in 0 seconds


In [ ]:
print("Get team stadiums html: ")
start_time = time.time()
nv = 17
for v in range(1,nv+1):
    team stadiums_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/9-stadiums/team stadiums_html_0.csv")
    get_html("teams", teams_html, "./drive/MyDrive/transfermarkt/9-stadiums/team stadiums_data", v)
print(f"Finished in {round(time.time()-start_time)} seconds")

```
Get team stadiums html: 
	Get team stadiums html 01: ==================================================  100.0%	|   715 seconds
	Get team stadiums html 02: ==================================================  100.0%	|   695 seconds
	Get team stadiums html 03: ==================================================  100.0%	|   685 seconds
	Get team stadiums html 04: ==================================================  100.0%	|   693 seconds
	Get team stadiums html 05: ==================================================  100.0%	|   682 seconds
	Get team stadiums html 06: ==================================================  100.0%	|   690 seconds
	Get team stadiums html 07: ==================================================  100.0%	|   695 seconds
	Get team stadiums html 08: ==================================================  100.0%	|   687 seconds
	Get team stadiums html 09: ==================================================  100.0%	|   684 seconds
	Get team stadiums html 10: ==================================================  100.0%	|   671 seconds
	Get team stadiums html 11: ==================================================  100.0%	|   675 seconds
	Get team stadiums html 12: ==================================================  100.0%	|   682 seconds
	Get team stadiums html 13: ==================================================  100.0%	|   692 seconds
	Get team stadiums html 14: ==================================================  100.0%	|   698 seconds
	Get team stadiums html 15: ==================================================  100.0%	|   699 seconds
	Get team stadiums html 16: ==================================================  100.0%	|   429 seconds
Finished in 10712 seconds
```

In [ ]:
def get_stadiums_url(ind):
    global team_stadiums_html
    team_stadium = team_stadiums_html.iloc[ind]
    team_id = team_stadium["id"]
    team_url = team_stadium["url"]
    if "ERROR 500" in team_stadium["content"]:
        return []
    soup = BeautifulSoup(team_stadium["content"], "lxml")
    df = []
    options = soup.find("select", {"id": "stadion_id"})
    if options is None:
        if soup.find('a', {'id': team_id}):
            name = soup.find('a', {'id': team_id}).text
            df.append([team_id, team_id, "", name, team_url])
    else:
        options = options.find_all("option")
        for opt in options:
            stadium_id = opt.get("value")
            name = opt.text
            url = f"{team_url}/stadion_id/{stadium_id}"
            df.append([f"{team_id}_{stadium_id}", team_id, stadium_id, name, url])
    return df

In [ ]:
print("Get stadiums url: ")
start_time = time.time()
nv = 16
for v in range(1,nv+1):
    team_stadiums_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/9-stadiums/team_stadiums_html_{v:03}.csv")
    p = Pool(20)
    stadiums_html = stadiums_html.append(pd.DataFrame(reduce(lambda a, b: a + b, \
                        p.map(get_stadiums_url, range(team_stadiums_html.shape[0]))), \
                        columns=["id","team_id","stadium_id","name","url"]), \
                        ignore_index=True)
    stadiums_html = stadiums_html.drop_duplicates(subset="url").reset_index(drop=True)
    p.terminate()
    p.join()
    p = 100*v/nv
    print(f"\r{'='*round(p/2)}{' '*(50-round(p/2))} {p:.2f}%", end="")
stadiums_html.to_csv("./drive/MyDrive/transfermarkt/9-stadiums/stadiums_html_0.csv", index=False)
print(f"\nFinished in {round(time.time() - start_time)} seconds")

Get stadiums url: 
================================================== 100.00%
Finished in 454 seconds


In [ ]:
stadiums_html.shape

(17299, 6)

In [ ]:
print("Get stadiums html: ")
nv = 18
start_time = time.time()
for v in range(1, nv+1):
    stadiums_html = pd.read_csv(f"./drive/MyDrive/transfermarkt/9-stadiums/stadiums_html_0.csv")
    get_html_large("stadiums", stadiums_html, "./drive/MyDrive/transfermarkt/9-stadiums/stadiums_html", v)
print(f"Finished in {round(time.time()-start_time)} seconds")

```
Get stadiums html: 
	Get stadiums html 01: ==================================================  100.0%	|   722 seconds
	Get stadiums html 02: ==================================================  100.0%	|   722 seconds
	Get stadiums html 03: ==================================================  100.0%	|   713 seconds
	Get stadiums html 04: ==================================================  100.0%	|   704 seconds
	Get stadiums html 05: ==================================================  100.0%	|   719 seconds
	Get stadiums html 06: ==================================================  100.0%	|   722 seconds
	Get stadiums html 07: ==================================================  100.0%	|   728 seconds
	Get stadiums html 08: ==================================================  100.0%	|   719 seconds
	Get stadiums html 09: ==================================================  100.0%	|   735 seconds
	Get stadiums html 10: ==================================================  100.0%	|   734 seconds
	Get stadiums html 11: ==================================================  100.0%	|   730 seconds
	Get stadiums html 12: ==================================================  100.0%	|   728 seconds
	Get stadiums html 13: ==================================================  100.0%	|   748 seconds
	Get stadiums html 14: ==================================================  100.0%	|   747 seconds
	Get stadiums html 15: ==================================================  100.0%	|   772 seconds
	Get stadiums html 16: ==================================================  100.0%	|   771 seconds
	Get stadiums html 17: ==================================================  100.0%	|   762 seconds
	Get stadiums html 18: ==================================================  100.0%	|   224 seconds
Finished in 12761 seconds
```

# The End